## Calculate Content Similarity Metric

In [1]:
import pandas as pd
import numpy as np
import os
from top2vec import Top2Vec

/home/doosti@chapman.edu/.conda/envs/ctopics/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-01 12:15:06.320440: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-01 12:15:06.405717: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-01 12:15:06.407465: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 12:15:09.671668: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
PATH = "/home/doosti@chapman.edu/projects/Facebook/top2vec/"
DATA_PATH = os.path.join(PATH,'data')
model_name = "top2vec_deeplearn_distiluse_notoken_2024-06-27.model"
model_path = os.path.join(DATA_PATH, model_name)
textdata_file = "data_all_text.csv"
textdata_path = os.path.join(DATA_PATH, textdata_file)
data_file = "pooled_us_aug2020.dta"
data_path = os.path.join(DATA_PATH, data_file)

In [13]:
# load model
model = Top2Vec.load(model_path)
topic_sizes, topic_nums = model.get_topic_sizes()
print(f"Number of topics: {len(topic_sizes)}")
# load text data
textdata = pd.read_csv(textdata_path, low_memory=False)
print(f"Text Data Size: {textdata.shape}")
# load data
data = pd.read_stata(data_path)
print(f"Data Size: {data.shape}")

Number of topics: 3936


/tmp/ipykernel_8505/741096147.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  textdata = pd.read_csv(textdata_path)


Text Data Size: (820099, 8)
Data Size: (220033, 151)


In [18]:
# sanity checks
# control for creator ids
print(f"Out of {data.shape[0]} rows, {data[data.creator_id.isin(textdata.creator_id)].shape[0]} rows have creator_id in textdata")
# control for sponosr ids
print(f"Out of {data[data.sponsored==1].shape[0]} rows, {data[(data.sponsored==1) & (data.sponsor_id.isin(textdata.creator_id))].shape[0]} rows have sponsor_id in textdata")
# control for current similarity metrics
print(f"Number of rows with non-missing similarity: {data[data.similarity.notnull()].shape[0]}")

Out of 220033 rows, 220033 rows have creator_id in textdata
Out of 34028 rows, 17337 rows have sponsor_id in textdata
Number of rows with non-missing similarity: 17273
